# $\alpha$=0.1

In [2]:
import torch
import numpy as np
import torchvision.transforms as transforms               # include image preprocess tools
from torchvision.datasets import CIFAR100        # for loading images from Pytorch CIFAR
from torch.utils.data import DataLoader
import timm
import detectors
from src.raps import raps_scores, raps_classification, eval_aps, split_data_set
from src.temperature_scaling import ModelWithTemperature

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load fine-tuned model
model = timm.create_model("resnet50_cifar100", pretrained=True)
model = model.to(device)

# reprocess the images from CIFAR
data_transform = transforms.Compose([
    transforms.ToTensor(),          # transfer to tensor
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2762))  # normalize
])
# load images from CIFAR10
dataset = CIFAR100(root="../../data", train=False, download=True, transform=data_transform)

# temperature scaling
temp_scal_loader = DataLoader(dataset, batch_size=32, shuffle=True)
model = ModelWithTemperature(model, temperature=4.85).to(device)
model.set_temperature(temp_scal_loader)

model.eval()

# The number of times the experiment is going to be repeated
num_runs = 10

# error rate
alpha = 0.1
lambda_reg = 0.1
k_reg = 1

# construct and evaluate repeatedly
all_avg_set_sizes = []
all_avg_coverages = []
print("RAPS Classification, Start!\n")
for i in range(num_runs):
    print(f"Running experiment {i+1}/{num_runs}...")

    # splite dataset
    calib_dataset, test_dataset = split_data_set(dataset, random_seed=i)

    # load data set respectively
    calib_loader = DataLoader(calib_dataset, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # calculate q_hat
    calib_scores, _ = raps_scores(model, calib_loader, alpha, lambda_reg, k_reg, device)
    t_cal = np.quantile(calib_scores, 1 - 0.1)  # calculate 1-alpha quantile
    print(f"t_cal = {t_cal}")

    # construct APS
    aps, aps_labels, true_labels = raps_classification(model, test_loader, t_cal, lambda_reg, k_reg, device)

    # evaluate APS
    avg_set_size, avg_coverage = eval_aps(aps_labels, true_labels)
    print(f"Average Prediction Set Size After APS in runs {i+1}: {avg_set_size}")
    print(f"Average Coverage Ratein runs {i+1}: {avg_coverage}\n")

    # record current result
    all_avg_set_sizes.append(avg_set_size)
    all_avg_coverages.append(avg_coverage)

# calculate the final average result
final_avg_set_size = np.mean(all_avg_set_sizes)
final_avg_coverage = np.mean(all_avg_coverages)
final_set_size_std = np.std(all_avg_set_sizes, ddof=0)
final_coverage_std = np.std(all_avg_coverages, ddof=0)

print(f"Final Average Prediction Set Size: {final_avg_set_size:.2f} ± {final_set_size_std:.2f}")
print(f"Final Average Coverage: {final_avg_coverage:.4f} ± {final_coverage_std:.4f}")

Files already downloaded and verified
Before temperature - NLL: 1.067, ECE: 0.086
Optimal temperature: 4.691
After temperature - NLL: 2.948, ECE: 0.673
RAPS Classification, Start!

Running experiment 1/10...
t_cal = 0.40386754870414737
Total set size: 15337
Total coverage sets: 4538
Total samples amount: 5000
Average Prediction Set Size After APS in runs 1: 3.0674
Average Coverage Ratein runs 1: 0.9076

Running experiment 2/10...
t_cal = 0.3998346805572511
Total set size: 15133
Total coverage sets: 4530
Total samples amount: 5000
Average Prediction Set Size After APS in runs 2: 3.0266
Average Coverage Ratein runs 2: 0.906

Running experiment 3/10...
t_cal = 0.3886567294597627
Total set size: 14715
Total coverage sets: 4487
Total samples amount: 5000
Average Prediction Set Size After APS in runs 3: 2.943
Average Coverage Ratein runs 3: 0.8974

Running experiment 4/10...
t_cal = 0.39517943859100346
Total set size: 14986
Total coverage sets: 4511
Total samples amount: 5000
Average Predict

# $\alpha$=0.2

In [3]:
# error rate
alpha = 0.2
lambda_reg = 0.1
k_reg = 3

# construct and evaluate repeatedly
all_avg_set_sizes = []
all_avg_coverages = []
print("RAPS Classification, Start!\n")
for i in range(num_runs):
    print(f"Running experiment {i+1}/{num_runs}...")

    # splite dataset
    calib_dataset, test_dataset = split_data_set(dataset, random_seed=i)

    # load data set respectively
    calib_loader = DataLoader(calib_dataset, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # calculate q_hat
    calib_scores, _ = raps_scores(model, calib_loader, alpha, lambda_reg, k_reg, device)
    t_cal = np.quantile(calib_scores, 1 - alpha)  # calculate 1-alpha quantile
    print(f"t_cal = {t_cal}")

    # construct APS
    aps, aps_labels, true_labels = raps_classification(model, test_loader, t_cal, lambda_reg, k_reg, device)

    # evaluate APS
    avg_set_size, avg_coverage = eval_aps(aps_labels, true_labels)
    print(f"Average Prediction Set Size After APS in runs {i+1}: {avg_set_size}")
    print(f"Average Coverage Ratein runs {i+1}: {avg_coverage}\n")

    # record current result
    all_avg_set_sizes.append(avg_set_size)
    all_avg_coverages.append(avg_coverage)

# calculate the final average result
final_avg_set_size = np.mean(all_avg_set_sizes)
final_avg_coverage = np.mean(all_avg_coverages)
final_set_size_std = np.std(all_avg_set_sizes, ddof=0)
final_coverage_std = np.std(all_avg_coverages, ddof=0)

print(f"Final Average Prediction Set Size: {final_avg_set_size:.2f} ± {final_set_size_std:.2f}")
print(f"Final Average Coverage: {final_avg_coverage:.4f} ± {final_coverage_std:.4f}")

Files already downloaded and verified
Before temperature - NLL: 1.071, ECE: 0.081
Optimal temperature: 4.691
After temperature - NLL: 2.952, ECE: 0.677
RAPS Classification, Start!

Running experiment 1/10...
t_cal = 0.12253235876560216
Total set size: 9443
Total coverage sets: 4023
Total samples amount: 5000
Average Prediction Set Size After APS in runs 1: 1.8886
Average Coverage Ratein runs 1: 0.8046

Running experiment 2/10...
t_cal = 0.12095132917165756
Total set size: 9285
Total coverage sets: 4007
Total samples amount: 5000
Average Prediction Set Size After APS in runs 2: 1.857
Average Coverage Ratein runs 2: 0.8014

Running experiment 3/10...
t_cal = 0.12351956516504288
Total set size: 9531
Total coverage sets: 4038
Total samples amount: 5000
Average Prediction Set Size After APS in runs 3: 1.9062
Average Coverage Ratein runs 3: 0.8076

Running experiment 4/10...
t_cal = 0.12150392681360245
Total set size: 9338
Total coverage sets: 4001
Total samples amount: 5000
Average Predicti

In [13]:
# error rate
alpha = 0.05
lambda_reg = 0.02
k_reg = 1

# construct and evaluate repeatedly
all_avg_set_sizes = []
all_avg_coverages = []
print("RAPS Classification, Start!\n")
for i in range(num_runs):
    print(f"Running experiment {i+1}/{num_runs}...")

    # splite dataset
    calib_dataset, test_dataset = split_data_set(dataset, random_seed=i)

    # load data set respectively
    calib_loader = DataLoader(calib_dataset, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # calculate q_hat
    calib_scores, _ = raps_scores(model, calib_loader, alpha, lambda_reg, k_reg, device)
    t_cal = np.quantile(calib_scores, 1 - alpha)  # calculate 1-alpha quantile
    print(f"t_cal = {t_cal}")

    # construct APS
    aps, aps_labels, true_labels = raps_classification(model, test_loader, t_cal, lambda_reg, k_reg, device)

    # evaluate APS
    avg_set_size, avg_coverage = eval_aps(aps_labels, true_labels)
    print(f"Average Prediction Set Size After APS in runs {i+1}: {avg_set_size}")
    print(f"Average Coverage Ratein runs {i+1}: {avg_coverage}\n")

    # record current result
    all_avg_set_sizes.append(avg_set_size)
    all_avg_coverages.append(avg_coverage)

# calculate the final average result
final_avg_set_size = np.mean(all_avg_set_sizes)
final_avg_coverage = np.mean(all_avg_coverages)
final_set_size_std = np.std(all_avg_set_sizes, ddof=0)
final_coverage_std = np.std(all_avg_coverages, ddof=0)

print(f"Final Average Prediction Set Size: {final_avg_set_size:.2f} ± {final_set_size_std:.2f}")
print(f"Final Average Coverage: {final_avg_coverage:.4f} ± {final_coverage_std:.4f}")

RAPS Classification, Start!

Running experiment 1/10...
t_cal = 0.3306817322969437
Total set size: 34203
Total coverage sets: 4774
Total samples amount: 5000
Average Prediction Set Size After APS in runs 1: 6.8406
Average Coverage Ratein runs 1: 0.9548

Running experiment 2/10...
t_cal = 0.30727898776531226
Total set size: 30589
Total coverage sets: 4747
Total samples amount: 5000
Average Prediction Set Size After APS in runs 2: 6.1178
Average Coverage Ratein runs 2: 0.9494

Running experiment 3/10...
t_cal = 0.2989882439374925
Total set size: 29433
Total coverage sets: 4730
Total samples amount: 5000
Average Prediction Set Size After APS in runs 3: 5.8866
Average Coverage Ratein runs 3: 0.946

Running experiment 4/10...
t_cal = 0.32574432939291015
Total set size: 33420
Total coverage sets: 4766
Total samples amount: 5000
Average Prediction Set Size After APS in runs 4: 6.684
Average Coverage Ratein runs 4: 0.9532

Running experiment 5/10...
t_cal = 0.322485250234604
Total set size: 32

# Result  

$\alpha$=0.1  
- Final Average **Prediction Set Size： 2.99**
- Final Average **Coverage: 90.13%**  

$\alpha$=0.2  
- Final Average **Prediction Set Size： 1.89**
- Final Average **Coverage: 80.12%**  

$\alpha$=0.05  
- Final Average **Prediction Set Size： 6.21**
- Final Average **Coverage: 94.85%**